In [7]:
import torch
from torchvision.models import resnet18

# ResNet-18 모델 선언
model = resnet18(num_classes = 4)

# 가중치 불러오기
weights_path = "resnet18.pth"  # 실제 가중치 파일의 경로로 변경하세요.
model.load_state_dict(torch.load(weights_path,  map_location=torch.device('cpu')))

# 모델을 평가 모드로 설정 (배치 정규화나 드롭아웃과 같은 레이어의 동작이 변경됩니다)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import io
import torch
from PIL import Image
import json
import flask
from flask import Flask, request, jsonify
import pickle
from torchvision import transforms
from torchvision.models import resnet18
import base64

app = Flask(__name__)


## device 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

## 모델 로딩
model = resnet18(num_classes = 4)
weights_path = "resnet18.pth"
model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))
model.eval()

## 데이터 변환 모듈 로드 (검증 데이터 용으로)
transform = transforms.Compose(
    [
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]
)

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        data = flask.request.data.decode('utf-8')
        s = io.StringIO(data)
        js = json.load(s)

        label = js['label'] ## 사용자가 만든 음식 라벨
        image = Image.open(io.BytesIO(base64.b64decode(js['image'].split(',')[-1]))).convert('RGB')
        input_data = transform(image).unsqueeze(0).to(device)

        outputs = model(input_data)
        top4_op = outputs.topk(4, dim=-1)[1][0].numpy().tolist()
        
        ### 모델이 예측한것과 레시피의 음식이 일치하는지 확인 (음식 라벨과 모델 예측값이 일치하는지 확인)
        # code 작성
        
        
        result = {
            "results" : top4_op
        }

        return flask.Response(
            response=json.dumps(result, ensure_ascii=False, indent='\t'),
            status=200,
            mimetype="application/json"
        )

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=4000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.30.1.45:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Oct/2023 10:07:06] "POST /recommend HTTP/1.1" 404 -


In [4]:
import torch

a = torch.randn([1, 5])
top_a = a.topk(3, dim=-1)

In [6]:
top_a[0]

tensor([[2.0953, 2.0667, 0.7237]])

In [9]:
top_a[1][0]

tensor([2, 1, 0])

In [ ]:
import io
import torch
from PIL import Image
import json
import flask
from flask import Flask, request, jsonify
import pickle
from torchvision import transforms
from torchvision.models import resnet18
import base64
import pandas as pd

app = Flask(__name__)


## device 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

## 모델 로딩
model = resnet18(num_classes = 4)
weights_path = "resnet18.pth"
model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))
model.eval()

## 데이터 변환 모듈 로드 (검증 데이터 용으로)
transform = transforms.Compose(
    [
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]
)

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        data = flask.request.data.decode('utf-8')
        s = io.StringIO(data)
        js = json.load(s)

        label = js['label'] ## 사용자가 만든 음식 라벨
        image = Image.open(io.BytesIO(base64.b64decode(js['image'].split(',')[-1]))).convert('RGB')
        input_data = transform(image).unsqueeze(0).to(device)

        outputs = model(input_data)
        top4_op = outputs.topk(4, dim=-1)[1][0].numpy().tolist()
        
        ### 모델이 예측한것과 레시피의 음식이 일치하는지 확인 (음식 라벨과 모델 예측값이 일치하는지 확인)
        # code 작성
        
        
        result = {
            "results" : top4_op
        }

        return flask.Response(
            response=json.dumps(result, ensure_ascii=False, indent='\t'),
            status=200,
            mimetype="application/json"
        )

    
# 레시피 추천 api
import ast

# 메뉴와 재료 가져오기

df = pd.read_csv('recipe.csv', encoding='euc-kr')
df.head()

cook_list = []
ingredients = []
all_ingredients = {} 
for idx, row in df.iterrows():
    cook_list.append(row['메뉴'])
    ingredient_map = {}
    for val in ast.literal_eval(row['재료']):
        ingredient_map[val] = 1
        all_ingredients[val] = 1
    ingredients.append(ingredient_map)

# 재료의 카테고리 번호를 저장 (중복X)
category_index = {} 
for idx, val in enumerate(sorted(all_ingredients.keys())):
    category_index[val] = idx

# 넘파이 어레이로 바꿔줌
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

matrix = []
for ingred_map in ingredients:
    tmp = []
    for val in ingred_map.keys(): 
        tmp.append(category_index[val])
    matrix.append(tmp)

# 0, 1 행렬로 변환
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
multi_hot_matrix = mlb.fit_transform(matrix)



@app.route('/recommend', methods=['POST'])
def recommend():
    if request.method == 'POST':
        data = flask.request.data.decode('utf-8')
        s = io.StringIO(data)
        js = json.load(s)
        
        ing = js['user_ing']
        
        all_ing_len = len(list(all_ingredients.keys()))
        
        user_matrix = [0 for i in range(all_ing_len)]
        for i in ing:
            user_matrix[i]=1
        
        similarity_scores = cosine_similarity([user_matrix], multi_hot_matrix)[0]
        
        idxs = np.argsort(similarity_scores)[::-1]

        ## 상위 5개 점수
        top5_scores = similarity_scores[idxs][:5]
        top5_idxs = idxs[:5] ## 상위 5개 인덱스 번호
        top5_recipe = [] ## 상위 5개 매칭 레시피

        for id in top5_idxs:
            top5_recipe.append(cook_list[id])
        
        result = {
            "recipe" : top5_recipe
        }
        
        return flask.Response(
            response=json.dumps(result, ensure_ascii=False, indent='\t'),
            status=200,
            mimetype="application/json"
        )

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=4000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.30.1.45:5500/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Oct/2023 10:39:12] "POST /recommend HTTP/1.1" 200 -


In [1]:
user_matrix = [0 for i in range(3)]

In [4]:
user_matrix[1]=1
user_matrix

[0, 1, 0]

In [4]:
ing = [118,
  35,
  57,
  121,
  91,
  9,
  172,
  19,
  158,
  36,
  99,
  149,
  109,
  178,
  44,
  144,
  82,
  104,
  114,
  38,
  155]

In [5]:
all_ing_len = 180
        
user_matrix = [0 for i in range(all_ing_len)]
for i in ing:
    user_matrix[i]=1
    

In [7]:
user_matrix[118]

1